<a href="https://colab.research.google.com/github/rer10013/ML-project/blob/main/Pytorch_Basic_3_4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# library
from torchvision import datasets
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Minst dataset
!git clone https://github.com/baek2sm/ml.git
!tar zxvf ml/datasets/MINST.tar.gz

# Load Minst dataset
path = './'
train_dataset = datasets.MNIST(path, train=True, download=True)
test_dataset = datasets.MNIST(path, train=False, download=True)

# Make train/test set
X_train, y_train = train_dataset.data /255, train_dataset.targets
X_test, y_test = test_dataset.data /255, test_dataset.targets

Cloning into 'ml'...
remote: Enumerating objects: 299, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 299 (delta 6), reused 3 (delta 3), pack-reused 289 (from 1)
Receiving objects: 100% (299/299), 64.38 MiB | 36.61 MiB/s, done.
Resolving deltas: 100% (155/155), done.
tar (child): ml/datasets/MINST.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


100%|██████████| 9.91M/9.91M [00:00<00:00, 15.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 450kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.20MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.86MB/s]


In [3]:
# Check sets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

torch.Size([60000, 28, 28])
torch.Size([60000])
torch.Size([10000, 28, 28])
torch.Size([10000])


In [4]:
# 2d to 3d
X_train, X_test = X_train.unsqueeze(1), X_test.unsqueeze(1)

# Check xsets
print(X_train.shape)
print(X_test.shape)

torch.Size([60000, 1, 28, 28])
torch.Size([10000, 1, 28, 28])


In [5]:
# Make dataset
train_dset = TensorDataset(X_train, y_train)
test_dset = TensorDataset(X_test, y_test)

# Make dataloader
train_loader = DataLoader(train_dset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dset, batch_size=32, shuffle=False)

In [6]:
# CNN class
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.hidden_layer1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)),
            nn.Dropout(0.5)
        )

        self.hidden_layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)),
            nn.Dropout(0.5)
        )

        self.hidden_layer3 = nn.Linear(128*5*5, 128)
        self.output_layer = nn.Linear(128, 10)

    def forward(self, X):
        out = self.hidden_layer1(X)
        out = self.hidden_layer2(out)
        out = out.view(out.size(0), -1)
        out = self.hidden_layer3(out)
        out = self.output_layer(out)
        return out

In [7]:
# Set environment
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [11]:
# Train model
def train(model, criterion, optimizer, loader):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()

        y_pred = model(X_batch)
        hypothesis = model(X_batch)
        loss = criterion(hypothesis, y_batch)
        loss.backward()
        optimizer.step()

        y_predicted = torch.argmax(hypothesis, 1)
        acc = (y_predicted == y_batch).float().mean()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(loader), epoch_acc / len(loader)

In [12]:
# Evaluation model
def evaluate(model, criterion, loader):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            hypothesis = model(X_batch)
            loss = criterion(hypothesis, y_batch)
            y_predicted = torch.argmax(hypothesis, 1)
            acc = (y_predicted == y_batch).float().mean()
            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(loader), epoch_acc / len(loader)

In [13]:
# Train
n_epochs = 20

for epoch in range(1, n_epochs + 1):
    train_loss, train_acc = train(model, criterion, optimizer, train_loader)
    valid_loss, valid_acc = evaluate(model, criterion, test_loader)

    print(f'Epoch: {epoch} Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}')

Epoch: 1 Train Loss: 0.0499, Train Acc: 0.9841 Valid Loss: 0.0283, Valid Acc: 0.9903
Epoch: 2 Train Loss: 0.0472, Train Acc: 0.9853 Valid Loss: 0.0303, Valid Acc: 0.9891
Epoch: 3 Train Loss: 0.0466, Train Acc: 0.9845 Valid Loss: 0.0275, Valid Acc: 0.9910
Epoch: 4 Train Loss: 0.0436, Train Acc: 0.9861 Valid Loss: 0.0276, Valid Acc: 0.9910
Epoch: 5 Train Loss: 0.0416, Train Acc: 0.9870 Valid Loss: 0.0275, Valid Acc: 0.9900
Epoch: 6 Train Loss: 0.0392, Train Acc: 0.9875 Valid Loss: 0.0259, Valid Acc: 0.9913
Epoch: 7 Train Loss: 0.0389, Train Acc: 0.9878 Valid Loss: 0.0256, Valid Acc: 0.9913
Epoch: 8 Train Loss: 0.0373, Train Acc: 0.9879 Valid Loss: 0.0250, Valid Acc: 0.9921
Epoch: 9 Train Loss: 0.0367, Train Acc: 0.9885 Valid Loss: 0.0243, Valid Acc: 0.9915
Epoch: 10 Train Loss: 0.0351, Train Acc: 0.9884 Valid Loss: 0.0247, Valid Acc: 0.9916
Epoch: 11 Train Loss: 0.0347, Train Acc: 0.9888 Valid Loss: 0.0237, Valid Acc: 0.9923
Epoch: 12 Train Loss: 0.0337, Train Acc: 0.9888 Valid Loss: 0.0